In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
from dataclasses import dataclass
from typing import List, Tuple, Dict, Callable, Optional
from pathlib import Path
from sklearn.metrics import r2_score
import scipy

# ============================================================== 
# 1. Define Basic Circuit Elements and Their Impedance Functions 
# ============================================================== 

def coth(x):
    return np.cosh(x) / np.sinh(x)

class CircuitElement:
    @staticmethod
    def R(omega: np.ndarray, R: float) -> np.ndarray:
        return np.full_like(omega, R, dtype=complex)
    
    @staticmethod
    def C(omega: np.ndarray, C: float) -> np.ndarray:
        # Add a small epsilon to avoid division by zero.
        return 1.0 / (1j * (omega + 1e-12) * C)
    
    @staticmethod
    def L(omega: np.ndarray, L: float) -> np.ndarray:
        return 1j * omega * L
    
    @staticmethod
    def CPE(omega: np.ndarray, Y0: float, n: float) -> np.ndarray:
        return 1.0 / (Y0 * (1j * omega) ** n)
    
    @staticmethod
    def W(omega: np.ndarray, Y0: float) -> np.ndarray:
        return 1.0 / (Y0 * np.sqrt(1j * omega))
    
    @staticmethod
    def Ws(omega: np.ndarray, Y0: float, B: float) -> np.ndarray:
        return 1.0 / (Y0 * np.sqrt(1j * omega) * np.tanh(B * np.sqrt(1j * omega)))
    
    @staticmethod
    def Wo(omega: np.ndarray, Y0: float, B: float) -> np.ndarray:
        return 1.0 / (Y0 * np.sqrt(1j * omega) * coth(B * np.sqrt(1j * omega)))
    
    @staticmethod
    def G(omega: np.ndarray, Y: float, k: float) -> np.ndarray:
        return 1.0 / (Y * np.sqrt(k + 1j * omega))

# Helper functions for combining impedances
def series_impedance(Z1: np.ndarray, Z2: np.ndarray) -> np.ndarray:
    return Z1 + Z2

def parallel_impedance(Z1: np.ndarray, Z2: np.ndarray) -> np.ndarray:
    # Avoid division by near-zero.
    denom = Z1 + Z2
    denom[np.abs(denom) < 1e-12] = 1e-12
    return (Z1 * Z2) / denom

# =======================================================
# 2. Define a Basic Element Specification for Candidate Trees
# =======================================================

@dataclass
class BasicElementSpec:
    name: str
    n_params: int
    param_names: List[str]
    param_bounds: List[Tuple[float, float]]
    initial_guess: List[float]
    impedance_fn: Callable[[np.ndarray, List[float]], np.ndarray]

# Create a dictionary of basic elements.
basic_elements: Dict[str, BasicElementSpec] = {
    "R": BasicElementSpec(
        name="R",
        n_params=1,
        param_names=["R"],
        param_bounds=[(1, 1e3)],
        initial_guess=[100],
        impedance_fn=lambda omega, p: CircuitElement.R(omega, p[0])
    ),
    "C": BasicElementSpec(
        name="C",
        n_params=1,
        param_names=["C"],
        param_bounds=[(1e-8, 1e-4)],
        initial_guess=[1e-6],
        impedance_fn=lambda omega, p: CircuitElement.C(omega, p[0])
    ),
    "L": BasicElementSpec(
        name="L",
        n_params=1,
        param_names=["L"],
        param_bounds=[(1e-6, 1e-2)],
        initial_guess=[1e-4],
        impedance_fn=lambda omega, p: CircuitElement.L(omega, p[0])
    ),
    "CPE": BasicElementSpec(
        name="CPE",
        n_params=2,
        param_names=["Y0", "n"],
        param_bounds=[(1e-6, 1e-2), (0.5, 1.0)],
        initial_guess=[1e-4, 0.8],
        impedance_fn=lambda omega, p: CircuitElement.CPE(omega, p[0], p[1])
    ),
    "W": BasicElementSpec(
        name="W",
        n_params=1,
        param_names=["Y0"],
        param_bounds=[(1e-6, 1e-2)],
        initial_guess=[1e-4],
        impedance_fn=lambda omega, p: CircuitElement.W(omega, p[0])
    ),
    "Ws": BasicElementSpec(
        name="Ws",
        n_params=2,
        param_names=["Y0", "B"],
        param_bounds=[(1e-6, 1e-2), (1e-6, 1e-2)],
        initial_guess=[1e-4, 1e-4],
        impedance_fn=lambda omega, p: CircuitElement.Ws(omega, p[0], p[1])
    ),
    "Wo": BasicElementSpec(
        name="Wo",
        n_params=2,
        param_names=["Y0", "B"],
        param_bounds=[(1e-6, 1e-2), (1e-6, 1e-2)],
        initial_guess=[1e-4, 1e-4],
        impedance_fn=lambda omega, p: CircuitElement.Wo(omega, p[0], p[1])
    ),
    "G": BasicElementSpec(
        name="G",
        n_params=2,
        param_names=["Y", "k"],
        param_bounds=[(1e-6, 1e-2), (1e-6, 1e-2)],
        initial_guess=[1e-4, 1e-4],
        impedance_fn=lambda omega, p: CircuitElement.G(omega, p[0], p[1])
    ),
}

# =======================================================
# 3. Represent a Candidate Circuit as a Tree (Series/Parallel)
# =======================================================

@dataclass
class CircuitCandidate:
    # For a leaf node, combination is None and element is one of the basic elements.
    combination: Optional[str] = None  # "series" or "parallel" or None for a leaf
    left: Optional['CircuitCandidate'] = None
    right: Optional['CircuitCandidate'] = None
    element: Optional[str] = None

    def get_param_info(self) -> Tuple[List[str], List[Tuple[float, float]], List[float]]:
        """Recursively collect parameter names, bounds, and initial guesses."""
        if self.combination is None:
            spec = basic_elements[self.element]
            return spec.param_names, spec.param_bounds, spec.initial_guess
        else:
            names_left, bounds_left, guess_left = self.left.get_param_info()
            names_right, bounds_right, guess_right = self.right.get_param_info()
            return names_left + names_right, bounds_left + bounds_right, guess_left + guess_right

    def compute_impedance(self, omega: np.ndarray, params: List[float]) -> np.ndarray:
        """
        Recursively compute the impedance for the candidate circuit.
        This method destructively consumes the list of parameters in order.
        """
        if self.combination is None:
            spec = basic_elements[self.element]
            n = spec.n_params
            p = params[:n]
            del params[:n]
            return spec.impedance_fn(omega, p)
        else:
            # First compute the impedance for the left subtree.
            Z_left = self.left.compute_impedance(omega, params)
            # Then compute the impedance for the right subtree with the remaining parameters.
            Z_right = self.right.compute_impedance(omega, params)
            if self.combination == "series":
                return series_impedance(Z_left, Z_right)
            elif self.combination == "parallel":
                return parallel_impedance(Z_left, Z_right)
            else:
                raise ValueError(f"Unknown combination type: {self.combination}")

    def canonical_repr(self) -> str:
        """
        Create a canonical string representation of the candidate circuit.
        For leaves, return the element name.
        For composites, sort the representations if the operation is commutative (parallel).
        """
        if self.combination is None:
            return self.element
        else:
            left_repr = self.left.canonical_repr()
            right_repr = self.right.canonical_repr()
            if self.combination == "parallel":
                parts = sorted([left_repr, right_repr])
                return f"parallel({parts[0]},{parts[1]})"
            else:  # series
                return f"series({left_repr},{right_repr})"

# =======================================================
# 4. Dynamic Programming to Generate All Candidate Circuits
#    with a Minimum of 3 and Maximum of 5 Basic Elements.
# =======================================================

def count_basic_elements(candidate: CircuitCandidate) -> int:
    """Count the number of basic (leaf) elements in the candidate circuit."""
    if candidate.combination is None:
        return 1
    else:
        return count_basic_elements(candidate.left) + count_basic_elements(candidate.right)

def generate_candidate_trees(max_elements: int) -> Dict[str, CircuitCandidate]:
    """
    Generate all unique candidate circuits (as CircuitCandidate trees)
    using between 1 and max_elements basic elements.
    Returns a dictionary mapping a canonical representation to the candidate.
    """
    dp: Dict[int, List[CircuitCandidate]] = {}
    dp[1] = [CircuitCandidate(combination=None, element=elem) for elem in basic_elements.keys()]
    
    all_candidates = {}  # Use a local dictionary to collect unique candidates.
    for cand in dp[1]:
        all_candidates[cand.canonical_repr()] = cand
    
    for n in range(2, max_elements + 1):
        dp[n] = []
        for k in range(1, n):
            for left_circuit in dp[k]:
                for right_circuit in dp[n - k]:
                    for comb in ["series", "parallel"]:
                        candidate = CircuitCandidate(
                            combination=comb,
                            left=left_circuit,
                            right=right_circuit
                        )
                        rep = candidate.canonical_repr()
                        if rep not in all_candidates:
                            all_candidates[rep] = candidate
        dp[n] = [c for rep, c in all_candidates.items() if count_basic_elements(c) == n]
    return all_candidates

# Generate candidate circuits using up to 5 basic elements,
# then filter to keep only those with at least 3 elements.
all_candidates = generate_candidate_trees(max_elements=5)
candidates = {rep: cand for rep, cand in all_candidates.items() if count_basic_elements(cand) >= 3}
print(f"Generated {len(candidates)} unique candidate circuits with 3 to 5 elements.")

# =======================================================
# 5. Data Processing, Fitting, and Candidate Selection
# =======================================================

class EISAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = None  # will hold the loaded data
        self.frequency = None
        self.omega = None
        self.real_Z = None
        self.imag_Z = None
        self.Z_measured = None
        self.load_data()
    
    def load_data(self) -> None:
        """
        Loads data from the file specified by self.file_path.
        """
        try:
            self.data = pd.read_excel(self.file_path)
        except Exception as e:
            print(f"Failed to load data: {e}")
    
    def process_data(self) -> None:
        """
        Processes the loaded data to extract frequency and impedance information.
        """
        # Filter out zero frequencies.
        freq = self.data["f (Hz)"].values
        self.frequency = freq[freq > 0]
        self.omega = 2 * np.pi * self.frequency
        self.real_Z = self.data["Re_Z (Ohm)"].values[:len(self.frequency)]
        self.imag_Z = self.data["Im_Z (Ohm)"].values[:len(self.frequency)]
        # For Nyquist plots, the imaginary part is taken as negative.
        self.Z_measured = self.real_Z + 1j * (-self.imag_Z)
    
    def calculate_error(self, Z_model: np.ndarray) -> Dict[str, float]:
        if np.any(np.isnan(Z_model)):
            return {'chi_squared': 1e20, 'R2_real': -np.inf, 'R2_imag': -np.inf}
        residuals = self.Z_measured - Z_model
        chi_squared = np.sum(np.abs(residuals) ** 2)
        try:
            r2_real = r2_score(self.real_Z, Z_model.real)
            r2_imag = r2_score(-self.imag_Z, Z_model.imag)
        except ValueError:
            r2_real = -np.inf
            r2_imag = -np.inf
        return {
            'chi_squared': chi_squared,
            'R2_real': r2_real,
            'R2_imag': r2_imag
        }
    
    
    def fit_candidate(self, candidate: CircuitCandidate) -> Optional[Dict]:
        param_names, param_bounds, initial_guess = candidate.get_param_info()
        
        def objective(params: np.ndarray) -> float:
            params_list = list(params)
            try:
                # Use a copy of the list so that the original is not modified externally.
                Z_model = candidate.compute_impedance(self.omega, params_list)
                if np.any(np.isnan(Z_model)) or np.any(np.abs(Z_model) > 1e12):
                    return 1e20
            except Exception:
                return 1e20
            return np.sum(np.abs(self.Z_measured - Z_model) ** 2)
        
        result = differential_evolution(
            objective,
            bounds=param_bounds,
            maxiter=1000,
            popsize=15,
            seed=42,
            polish=True,
            init="latinhypercube"  # This ensures the initial population is within the bounds.
        )
        if not result.success:
            return None
        
        params_opt = result.x
        # Re-compute the fitted impedance (make a copy of the parameter list)
        Z_fitted = candidate.compute_impedance(self.omega, list(params_opt))
        error_metrics = self.calculate_error(Z_fitted)
        
        return {
            'params': params_opt,
            'error': result.fun,
            'Z_fitted': Z_fitted,
            'metrics': error_metrics,
            'param_names': param_names,
            'structure': candidate.canonical_repr()
        }

    def find_best_candidate(self, candidate_dict: Dict[str, CircuitCandidate]) -> Tuple[str, Dict]:
        results = {}
        for name, candidate in candidate_dict.items():
            print(f"Fitting candidate circuit {name} ...")
            result = self.fit_candidate(candidate)
            if result:
                results[name] = result
                print(f"  Chi-squared: {result['error']:.2e}, R² (Real): {result['metrics']['R2_real']:.4f}")
                print("  Fitted parameters:")
                for pname, pval in zip(result['param_names'], result['params']):
                    print(f"    {pname} = {pval:.4g}")
            else:
                print("  Fit failed.")
        if not results:
            raise ValueError("No candidate circuits could be fitted successfully.")
        best_candidate = min(results.items(), key=lambda x: x[1]['error'])
        return best_candidate
    
    def plot_results(self, best_result: Dict, candidate_name: str) -> None:
        Z_fitted = best_result['Z_fitted']
        plt.figure(figsize=(12, 6))
        
        # Nyquist Plot
        plt.subplot(121)
        plt.plot(self.real_Z, -self.imag_Z, 'ko', label='Measured', alpha=0.6)
        plt.plot(Z_fitted.real, -Z_fitted.imag, 'r-', label='Fitted')
        plt.xlabel('Z_real (Ohm)')
        plt.ylabel('-Z_imag (Ohm)')
        plt.title('Nyquist Plot')
        plt.legend()
        plt.grid(True)
        
        # Bode Magnitude Plot
        plt.subplot(222)
        plt.loglog(self.frequency, np.abs(self.Z_measured), 'ko', alpha=0.6, label='Measured')
        plt.loglog(self.frequency, np.abs(Z_fitted), 'r-', label='Fitted')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('|Z| (Ohm)')
        plt.title('Bode Magnitude Plot')
        plt.legend()
        plt.grid(True)
        
        # Bode Phase Plot
        plt.subplot(224)
        phase_measured = np.degrees(np.arctan2(-self.imag_Z, self.real_Z))
        phase_fitted = np.degrees(np.arctan2(-Z_fitted.imag, Z_fitted.real))
        plt.semilogx(self.frequency, phase_measured, 'ko', alpha=0.6, label='Measured')
        plt.semilogx(self.frequency, phase_fitted, 'r-', label='Fitted')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Phase (degrees)')
        plt.title('Bode Phase Plot')
        plt.legend()
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()

# =======================================================
# 6. Main Execution
# =======================================================
if __name__ == "__main__":
    # Initialize analyzer with data file
    file_path = "Dataset1-exergy-r1--r2c1.xlsx"
    analyzer = EISAnalyzer(file_path)
    # Process the loaded data
    analyzer.process_data()

    # Fit all candidate circuits
    all_results = {}
    
    for name, candidate in candidates.items():
        print(f"\n{'='*60}\nFitting candidate circuit: {name}\n{'='*60}")
        result = analyzer.fit_candidate(candidate)
        if result:
            all_results[name] = result

    if not all_results:
        print("No valid solutions found!")
        exit()

    # Calculate error threshold (e.g., 1.2 times the minimum error)
    errors = [res['error'] for res in all_results.values()]
    min_error = min(errors)
    ERROR_THRESHOLD = 1.2 * min_error  # Adjust multiplier as needed
    
    # Filter and sort solutions
    filtered_results = {
        name: res for name, res in all_results.items() 
        if res['error'] <= ERROR_THRESHOLD
    }
    sorted_results = sorted(filtered_results.items(), key=lambda x: x[1]['error'])

    # Print header
    print(f"\n\n{'#'*80}")
    print(f" BEST SOLUTIONS (Error <= {ERROR_THRESHOLD:.2e})")
    print(f"{'#'*80}")

    # Print qualifying solutions
    for idx, (cand_name, result) in enumerate(sorted_results, 1):
        print(f"\n{'*'*60}")
        print(f"Solution #{idx}: {result['structure']}")
        print(f"  Chi-squared Error: {result['error']:.2e}")
        print(f"  R² (Real): {result['metrics']['R2_real']:.4f}")
        print(f"  R² (Imag): {result['metrics']['R2_imag']:.4f}")
        print("\n  Parameters:")
        for pname, pval in zip(result['param_names'], result['params']):
            print(f"    {pname:5} = {pval:.4g}")
        print('*'*60)

    # Plot best solution
    if sorted_results:
        best_name, best_result = sorted_results[0]
        print(f"\n\n{'#'*80}")
        print(f" PLOTTING BEST SOLUTION: {best_name}")
        print(f"{'#'*80}")
        analyzer.plot_results(best_result, best_name)
    else:
        print("\nNo solutions met the error threshold!")

    # Print final summary
    print(f"\n{'#'*80}")
    print(f" Processed {len(all_results)} candidates")
    print(f" Found {len(filtered_results)} solutions within error threshold")
    print(f" Minimum error: {min_error:.2e}")
    print(f"{'#'*80}")


Generated 2538650 unique candidate circuits with 3 to 5 elements.

Fitting candidate circuit: series(R,series(R,R))

Fitting candidate circuit: parallel(R,series(R,R))

Fitting candidate circuit: series(R,parallel(R,R))

Fitting candidate circuit: parallel(R,parallel(R,R))

Fitting candidate circuit: series(R,series(R,C))

Fitting candidate circuit: parallel(R,series(R,C))

Fitting candidate circuit: series(R,parallel(C,R))

Fitting candidate circuit: parallel(R,parallel(C,R))

Fitting candidate circuit: series(R,series(R,L))

Fitting candidate circuit: parallel(R,series(R,L))

Fitting candidate circuit: series(R,parallel(L,R))

Fitting candidate circuit: parallel(R,parallel(L,R))

Fitting candidate circuit: series(R,series(R,CPE))

Fitting candidate circuit: parallel(R,series(R,CPE))

Fitting candidate circuit: series(R,parallel(CPE,R))

Fitting candidate circuit: parallel(R,parallel(CPE,R))

Fitting candidate circuit: series(R,series(R,W))

Fitting candidate circuit: parallel(R,serie